In [ ]:
from transformers import MimiModel, AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi")
model = MimiModel.from_pretrained("kyutai/mimi")
model = model.to("cuda")

In [13]:

import torch
import torchaudio

test_arr = np.load("../../fish-speech.rs/out.npy")
test_input = torch.from_numpy(test_arr[:,:200]).to("cuda").to(torch.long).unsqueeze(0)

# test_input = torch.randint(low=0, high=2048, size=(8, 175)).unsqueeze(0).to("cuda")
out_pcm = model.decode(test_input)
torchaudio.save("out.wav", out_pcm.audio_values[0].to("cpu"), 24000)